In [0]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
#importing libraries
import numpy as np
import pandas as pd
import math

from scipy.spatial import distance
from collections import Counter
from sklearn.model_selection import train_test_split
from scipy import stats
import seaborn as sns


In [0]:
#uploading training data, test data to run the decision tree classifier 
#and test_label to calculate r2 score
training_data_dataFrame = pd.read_csv('/content/drive/My Drive/q3/train.csv')
test_data_dataFrame=pd.read_csv('/content/drive/My Drive/q3/test.csv')
test_label_dataFrame = pd.read_csv('/content/drive/My Drive/q3/test_labels.csv',header=None)



In [0]:
#Before Preprocessing copying the DataFrame to new DataFrame
dataFrameForClean = training_data_dataFrame.copy()
column_names = dataFrameForClean.columns


In [0]:
#PreProcessing Step 1: find all the columns where null values is more than 80% of training
#data
drop_columns=[]
for i in column_names:
    if dataFrameForClean[i].isnull().sum() > 800:
      drop_columns.append(i)
      # dataFrameForClean.drop([i],axis=1,inplace=True)


In [0]:
#PreProcessing Step 2: drop all the columns where null values is more than 80% of training
#data
dataFrameForClean.drop(drop_columns,axis=1,inplace=True)


In [0]:
#PreProcessing Step 3: drop all the columns where null values is more than 80%, 
# we have dropped columns from training data so we need to drop from training data as wee
test_data_dataFrame.drop(drop_columns,axis=1,inplace=True)

In [0]:
#find all the categorial columns and save it in (categorial_columns)
categorial_columns=dataFrameForClean.dtypes[dataFrameForClean.dtypes == "object"].index


In [0]:
#Now we have two Dataframe , one for training and other for testing 
#save all the categorial and Numerical variable of both Dataframe
#After that replace NONE in the data Frame with "nan" value
#Now for the categorial columns , replace it with mode
#Now for the numerical columns , repplace it with mean


categorial_columns=dataFrameForClean.dtypes[dataFrameForClean.dtypes == "object"].index
test_data_categorial_columns=test_data_dataFrame.dtypes[test_data_dataFrame.dtypes == "object"].index

#now find the numerical features
numerical_columns=dataFrameForClean.dtypes[dataFrameForClean.dtypes != "object"].index
test_data_numerical_columns=test_data_dataFrame.dtypes[test_data_dataFrame.dtypes != "object"].index

dataFrameForClean.replace('None',np.nan,inplace=True)
test_data_dataFrame.replace('None',np.nan,inplace = True)



for col in categorial_columns:
  dataFrameForClean.fillna(dataFrameForClean.mode().iloc[0],inplace=True)
  

for col in test_data_categorial_columns:
  test_data_dataFrame.fillna(test_data_dataFrame.mode().iloc[0],inplace=True)




for col in numerical_columns:
  dataFrameForClean.fillna(dataFrameForClean.mean(),inplace=True)
  

for col in test_data_numerical_columns:
  test_data_dataFrame.fillna(test_data_dataFrame.mean(),inplace=True)

In [0]:
#finding columns whose covariance with sales price is <0.4
#source: kaggle
#and saving [col_name,covar] in columnDelBecauseOfCoVar
#saving only col value in columnNeedToDelBsOfCoVar
columnDelBecauseOfCoVar=[]
columnNeedToDelBsOfCoVar=[]
for col in numerical_columns:
  if(col == 'Id'):
     continue
  if dataFrameForClean[col].corr(dataFrameForClean['SalePrice']) < 0.4:
    #print(col)
    #print(dataFrameForClean[col].corr(dataFrameForClean['SalePriceAfterLog']));
    columnDelBecauseOfCoVar.append([col,dataFrameForClean[col].corr(dataFrameForClean['SalePrice'])])
    columnNeedToDelBsOfCoVar.append(col)

In [0]:
# for col in columnNeedToDelBsOfCoVar:
  dataFrameForClean.drop(columnNeedToDelBsOfCoVar,axis=1,inplace=True)
  test_data_dataFrame.drop(columnNeedToDelBsOfCoVar,axis=1,inplace=True)

In [0]:
#saving the non categorial values of data frame in list
numerical_columns=dataFrameForClean.dtypes[dataFrameForClean.dtypes != "object"].index


In [0]:
# the below function give the value of categorial function at which we need to split 
#
def get_msr_for_categorical(col, dataFrame):
  uniqueValues = dataFrame[col].unique()
  col_specific_value=None
  col_value=0
  prev_msr=math.inf
  for feature in uniqueValues:
    same=dataFrameForClean[dataFrameForClean[col]==feature]
    large=dataFrameForClean[dataFrameForClean[col]!=feature]
    # print('same:',same.shape)
    # print('large',large.shape)
    same = same.to_numpy()
    large = large.to_numpy()
    A=same[:,[-1]]
    B=large[:,[-1]]
    
    msr1=0
    msr2=0
    msr1=((A-np.mean(A))**2)
    msr2=((B-np.mean(B))**2)
    

    

    total = same.shape[0] + large.shape[0]
    wmsr = int((sum(msr1)*len(same) + sum(msr2)*len(large))/total)
    # print('wmsr:',wmsr,':feature:',feature)
    
    if((wmsr)<prev_msr):
      prev_msr = wmsr
      col_specific_value = feature

  # print(type(prev_msr),prev_msr)
  return prev_msr , col_specific_value



In [0]:
# for the column containing non-categorial values , we are finding 
#msr value for each point in integer value 
#this function is a helper function for
#get_msr_for_int(col,data_frame)
def get_msr_for_int_mid_value(mid_value,col,data_frame):
  # data_frame = data_frame.to_numpy()
  small_values_dataFrame = data_frame[data_frame[col]<mid_value]
  
  large_value_dataFrame = data_frame[data_frame[col]>=mid_value]
  
  large_value_dataFrame=large_value_dataFrame.to_numpy()
  
  small_values_dataFrame=small_values_dataFrame.to_numpy()

  # print('large_value',len(large_value_dataFrame))
  # print('small_value',len(small_values_dataFrame))
  
  B=large_value_dataFrame[:,[-1]]
  
  A=small_values_dataFrame[:,[-1]]
  
  msr1=0
  msr2=0


  
  msr1=((A-np.mean(A))**2)
  msr2=((B-np.mean(B))**2)
    

    

  total = small_values_dataFrame.shape[0] + large_value_dataFrame.shape[0]
  wmsr = int((sum(msr1)*len(small_values_dataFrame) + sum(msr2)*len(large_value_dataFrame))/total)
    
 
  return (wmsr);


In [0]:
#The below function return the msr for non-categorial columns,the value at  
#which we need to split
def get_msr_for_int(col,data_frame):
  uniqueValues = data_frame[col].unique()
  
  uniqueValues.sort()

  prev_msr = math.inf
  
  prev_mean =None
  
 # for i in range(len(uniqueValues)):
  for i in range(len(uniqueValues)-1):

    (mid_value) = float(uniqueValues[i] + uniqueValues[i+1] ) /2
    # print('mid_value',mid_value)
    curr_msr = get_msr_for_int_mid_value(mid_value,col,data_frame)

    # print('curr_msr:',curr_msr,':prev_msr ',prev_msr)
    if curr_msr < prev_msr:
      
      prev_msr = curr_msr
      prev_mean = mid_value

  # print('get_msr_for_int','prev_msr',prev_msr,'prev_mean',prev_mean,'col' , col)
  return prev_msr , prev_mean , col



In [0]:
#it iterates in all the columns for the data Frame
#and find the value at which we need to split,and return the column name 
#in case of both categorial and non-categorial
def get_col_to_split(dataFrame):
  prev_msr = math.inf
  value=None
  col_value=None
  
  for col in dataFrame.columns:
    if col == "SalePrice" or col == "Id":
      continue
    
    if(dataFrame[col].dtype == "object"):
      curr_msr , mean_value = get_msr_for_categorical(col,dataFrame)
      #print('get_col_to_split,cate',curr_msr , mean_value)
    else:
      curr_msr , mean_value ,col = get_msr_for_int(col,dataFrame)
      #print('get_col_to_split,int',curr_msr , mean_value,col)
    #print(curr_msr , mean_value , col)  

    if(curr_msr < prev_msr):
      prev_msr = curr_msr
      value = mean_value
      col_value = col
      #print(col_value , value )
     

    
  #if(dataFrameForClean[col_value].dtypes == 'Object'):
  return col_value , value


In [0]:
#The below function splits the dataFrame into two 
#In case of int ,one for less than splitting value and other for greater that equal to 
#splitting value 
#In case of categorial, one for equal to splitting value and Other for not equal to 
#splitting value
def get_data_frame(training_data_build,splitting_value,col):
  print('training_data_build',training_data_build.shape[0])
  if dataFrameForClean[col].dtype == "object":
     mask = training_data_build[col] == splitting_value
     training_data_small = training_data_build[mask]
     training_data_large = training_data_build[~mask]
     print('training_data_small',training_data_small.shape[0],'training_data_large',training_data_large.shape[0])
     return training_data_small , training_data_large
  else:
    mask = training_data_build[col] < splitting_value
    training_data_small = training_data_build[mask]
    training_data_large = training_data_build[~mask]
    print('training_data_small',training_data_small.shape[0],'training_data_large',training_data_large.shape[0])
    return training_data_small , training_data_large


In [0]:
#structure of tree node
class Node:
  def __init__(self):
    self.left = None
    self.right = None
    self.leaf = 0
    self.splitting_value = None
    self.col  = None


In [0]:
#The function builds the tree , the depth of tree not to give as user argument 
#the minimum size of dataFrae alse,these two act as a base condition for recursion

def build_tree(training_data_build, max_depth, min_size,depth):
  root = Node()
  if depth > max_depth:
    splitting_value=training_data_build['SalePrice'].mean();
    root.leaf = 1
    root.splitting_value = splitting_value;
    # print('depth',splitting_value)
    # print('@'*30)
    # print(training_data_build.shape[0])
    # print('@'*30)
    return root

  if training_data_build.shape[0] < 20:
    splitting_value=training_data_build['SalePrice'].mean();
    root.leaf = True
    root.splitting_value = splitting_value;
    # print('size',splitting_value)
    # print('@'*30)
    # print(training_data_build.shape[0])
    # print('@'*30)
    return root

  col , splitting_value = get_col_to_split(training_data_build)
  training_data_small , training_data_large  = get_data_frame(training_data_build,splitting_value,col)
  print('splitting',splitting_value,' col:',col)
  print('small',training_data_small.shape[0])
  print('large',training_data_large.shape[0])
  del training_data_small[col]
  del training_data_large[col]

  #print(splitting_value)
  
  
  
  root.splitting_value = splitting_value
  root.col = col
  # print(splitting_value,'-',col,'-',depth)
  
  root.left = build_tree(training_data_small,max_depth,min_size,depth + 1)

  root.right =  build_tree(training_data_large,max_depth,min_size,depth +1)
  print('root.spli:',root.splitting_value,':root:',root.col,':depth:',depth,'leaf:',root.left)
  return root


In [0]:
#creates the tree and return the root address
root1 = build_tree(dataFrameForClean,6,0,0)

In [0]:
#function to pint the inorder
def printInorder(root1,depth):
  if root1:
    print(root1.splitting_value,root1.col,root1.leaf,depth)
    printInorder(root1.left,depth+1)
    printInorder(root1.right,depth+1)
    
   
    

In [0]:
#find_ans is a prediction function , to find the predicted value correspond to 
#test case
def find_ans(row,root1):
  
  if root1.left==None and root1.right==None:
    print('leaf',root1.splitting_value)
    return root1.splitting_value
  print('root1.col',root1.col,dataFrameForClean[root1.col].dtypes)
  # print ( 'compare between:', dataFrameForClean[root1.col] , '!!!!!!', row[root1.col],' ==', root1.splitting_value)
  if(dataFrameForClean[root1.col].dtypes == "object"):
    
    if(row[root1.col] == root1.splitting_value):
      print('going left',root1.splitting_value)
      
      return find_ans(row,root1.left)
    else:
      print('going right',root1.splitting_value)
      return find_ans(row,root1.right)
  else:
    
    if(row[root1.col] < root1.splitting_value):
      print('g t left',root1.splitting_value)
      return find_ans(row,root1.left)
    
    else:
      print('g to right',root1.splitting_value)
      return find_ans(row,root1.right)



In [0]:
#final answer is a list , which contains all the predicted values corresponding to 
#test file
final_answer=[]

for i in range(0,len(test_data_dataFrame)):
  final_answer.append(find_ans(test_data_dataFrame.iloc[i],root1))


In [0]:
#Reading the test_label_dataFrame file to verify it
test_label_dataFrame = pd.read_csv('/content/drive/My Drive/q3/test_labels.csv',header=None)


In [0]:
#dropping the 1st column from test_label_file , to give it as a argument to 
#r2 function
test_label_dataFrame.drop(test_label_dataFrame.columns[0], axis=1,inplace = True)

In [0]:
#converting to numpy array
test_label_dataFrame= test_label_dataFrame.to_numpy()

In [0]:
from sklearn.metrics import r2_score

In [0]:
#R-squared (R2) is a statistical measure that represents the proportion of the variance for a 
#dependent variable that's explained by an independent variable or variables in a regression model
# the higher the R-squared, the better the model fits your data.
r2_score(test_label_dataFrame,final_answer)

0.7504721386291735